In [1]:
from src.routing import create_data, get_options_df, solve, prelim_check, save_results, load_results, append_and_save_new_results
import time
import pandas as pd
import numpy as np

# 1. Data Input

## Work Schedule

### MAIN

In [2]:
# might change week to week / problem by problem (when recalculating)
work_schedule = {
    1: [7 * 60, 19 * 60],
    2: [8 * 60, 18 * 60],
    3: [18 * 60, 18 * 60],
    4: [19 * 60, 19 * 60],
    5: [17 * 60, 17 * 60],
    6: [18 * 60, 18 * 60],
    7: [18 * 60, 18 * 60]
}
start_node = 0 # define only for dynamic recalculating for a single day
overnight_stays_ok = {1}

# stable parameters
min_distance_overnight = 50 # will be used to determine if an overnight stay is necessary even when planned
lunch_earliest_start = 12 # could be derived per day from work_schedule
lunch_duration = 30 # will need to be set to 0 if lunch is not mandatory
fix_app_margin = 10 # min number of minutes to arrive before appointments

# dummy data parameters
num_nodes = 100
min_work_days = 5
percentage_of_appointments = 0

### DERIVED

In [3]:
day_lengths = {day: (end - start) for day, (start, end) in work_schedule.items()}
days_off = {day for day, length in day_lengths.items() if length == 0}
work_days = {day for day in day_lengths if day not in days_off}
max_day_length = max(day_lengths.values())
relative_day_lengths = [round(length / max_day_length, 2) for _, length in day_lengths.items()]
no_overnight_stays = {day for day in day_lengths if day not in days_off and day not in overnight_stays_ok}
max_days_off = len(days_off)
n_work_days = 7 - max_days_off
if start_node != 0 & n_work_days > 0:
    # throw a warning
    print("Warning: Start node is not the home node, but work days are still available.")
    start_node = 0

## Algorithm (Developer Settings)

In [4]:
overnight_cost = 2 # defintion of cost for overnight stays
GlobalSpanCostCoefficient = 0 # set to 0 to focus on adding more nodes rather than reducting trip lengths/durations
slack = 2000 # slack for the time windows
num_nodes_metric_coef = 2
spread_metric_coef = 1
num_fixed_metric_coef = 0
doubling_exp = 0.8 # 1 = double the length, double the size/spread/num_fixed; 0.5 = less than double; 2 = more than double
cluster_sizes_below_1 = 1 # should be smaller the larger the doubling_exp is since small clusters will be smaller / large ones larger
penalties = [5000, 2500, 1000, 500, 100] # first one for fixed appointments then week 0, 1, 2, 3
first_algorithm = 'SAVINGS' # 'PATH_CHEAPEST_ARC', 'SAVINGS', 'PARALLEL_CHEAPEST_INSERTION', 'BEST_INSERTION'
second_algorithm = 'GUIDED_LOCAL_SEARCH' # 'GUIDED_LOCAL_SEARCH', 'GREEDY_DESCENT'

# 2. Dummy data creation and checks

In [5]:
nodes_df, time_matrix = create_data(num_nodes, percentage_of_appointments, min_work_days, days_off, penalties, home_node_id=0, visiting_interval_min=10, visiting_interval_max=30, max_last_visit=30, simple_schedule=False)
nodes_df, messages = prelim_check(nodes_df, time_matrix, work_schedule, fix_app_margin, work_days, verbose=False)
options_df = get_options_df(days_off, relative_day_lengths, no_overnight_stays, max_days_off)
messages

['Appointments not within the work schedule: []',
 'Appointments not within opening hours of node: []',
 'Appointments removed due to conflicts among fixed appointments: []']

# 3. Metadata - collection 1 (input)

In [6]:
# Collect total hours, fixed appointments, spread and number_of_possible_soluions
total_hours = 0
for day, hours in work_schedule.items():
    if not day in days_off:
        total_hours += hours[1] - hours[0]

n_fixed_appointments = sum(nodes_df['fixed_appointment'].notnull())

avg_spread = time_matrix.iloc[nodes_df['node_id'].values, nodes_df['node_id'].values].median().median()
max_spread = time_matrix.iloc[nodes_df['node_id'].values, nodes_df['node_id'].values].max().max()

number_of_possible_solutions = len(options_df)

# 4. Automatic Parameter Optimization (APO)

In [7]:
# Dependent on total work hours, number of fixed appointments, geopgraphic spread and number of solutions to compare (in future, this could try definint optimal solution beforehand)
time_limit = 10
num_nodes_to_consider = 8

# 5. Defining nodes to be considered in current week

In [8]:
nodes_df = nodes_df.sort_values('priority', ascending=False, inplace=False)
top_nodes_df = nodes_df.head(num_nodes_to_consider)
if 0 not in top_nodes_df['node_id'].values:
    node_zero_df = nodes_df[nodes_df['node_id'] == 0]
    nodes_df = pd.concat([node_zero_df, top_nodes_df]).head(num_nodes_to_consider)
else:
    nodes_df = top_nodes_df
nodes_df = nodes_df.sort_values('node_id', ascending=True, inplace=False)

# 6. Metadata - storage loading

In [9]:
existing_results_df = load_results()
max_index = existing_results_df['index'].max() if len(existing_results_df) > 0 else 0
results = []

# 7. Routing

In [10]:
# Main
start_time = time.time()

updated_options_df, figs, routes, updated_nodes_df = solve(options_df, nodes_df, time_matrix, slack, work_schedule, fix_app_margin, time_limit, lunch_duration, first_algorithm, second_algorithm, GlobalSpanCostCoefficient, min_distance_overnight, start_node, num_nodes_metric_coef, spread_metric_coef, num_fixed_metric_coef, doubling_exp, cluster_sizes_below_1, verbose=False, visual=1, restrictive=False)

end_time = time.time()
duration = end_time - start_time

# Define best option automatically
max_app_cons = updated_options_df['fixed_app_cons'].max()
best_solutions = updated_options_df[updated_options_df['fixed_app_cons'] == max_app_cons]
best_solutions['num_nodes_cons'] -= best_solutions['n_overnight_trips'] * overnight_cost
best_solution = best_solutions['num_nodes_cons'].idxmax()
best_option_df = updated_options_df.iloc[best_solution]

Option 0 with objective value 677 and 0 dropped nodes


Option 1 with objective value 522 and 0 dropped nodes


In [14]:
def extract_visited_nodes_by_day(routes_dict):
    day_to_nodes = {}  # Initialize a dictionary to store sets of nodes for each day.
    
    for key, value in routes_dict.items():
        # Extract the day from the route key, assuming it's within curly braces and is an integer.
        day = int(key[key.find('{') + 1:key.find('}')])
        if day not in day_to_nodes:
            day_to_nodes[day] = set()
        
        # Iterate over the tuples in the list (node_id, timestamp)
        for node_id, _ in value:
            if node_id != 0:  # Exclude node 0
                day_to_nodes[day].add(node_id)
    
    return day_to_nodes

routes_dict = best_option_df['route_lists']
extract_visited_nodes_by_day(routes_dict)

# define fixed appointment visits and define how a visit there might require different updating of subsequent last visit dates in nodes df

{2: {54, 57, 62, 63}, 1: {56, 59, 60}}

# Recursion

In [12]:
# collect respected nodes to upate last visit
[x for x in list(best_option_df['respected_nodes']) if x != 0]

[54, 56, 57, 59, 60, 62, 63]

# 9. Metadata - collection 2 (output)

In [13]:
# Collect results
result = {
    "index": max_index + 1,
    "time": pd.Timestamp.now().strftime('%Y-%m-%d %H:%M'),
    "in_first_algo": first_algorithm,
    "in_second_algo": second_algorithm,

    "in_fixed_app_fraction": percentage_of_appointments,

    "thr_num_fixed_appointments": n_fixed_appointments,
    "thr_total_hours": total_hours,
    "thr_avg_spread": round(avg_spread, 2),
    "thr_max_spread": round(max_spread, 2),
    "thr_n_work_days": n_work_days,
    "thr_number_of_possible_solutions": number_of_possible_solutions,

    "thr_time_limit": time_limit,
    "thr_num_nodes_to_consider": num_nodes_to_consider,

    "out_obj_value": best_option_df["obj_value"],
    "out_num_nodes_cons": best_option_df['num_nodes_cons'],
    "out_n_overnight_trips": best_option_df['n_overnight_trips'],
    "out_rel_nodes_cons": round(best_option_df['nodes_cons'],2),
    "out_rel_fixed_app_cons": best_option_df['fixed_app_cons'] if len(updated_options_df['fixed_appointments_nodes'].values[0]) >= 1 else np.nan,
    "out_time_taken": round(duration, 2)
}
results.append(result)

# Convert list of dictionaries to DataFrame
new_results_df = pd.DataFrame(results)

append_and_save_new_results(new_results_df)

Results saved to vrp_results.csv
